## Birdsongs - 01.- Obteniendo Datos - WebScraping Dataset


A partir de una web recuperamos el dataset con las grabaciones que van a formar parte de nuestro estudio. El resultado de este notebook será un fichero csv con el detalle de las grabaciones con las que iniciaremos el trabajo.

## a) Selección del dataset

La obtención del dataset se realiza a través de la web [**Xeno-Canto**](https://www.xeno-canto.org/). Es una web dedicada a compartir cantos de aves de todo el mundo e invita a investigadores, aficionados y cualquier persona interesada en las aves, a escuchar, descargar y explorar su colección de cantos.



![xenocanto](./resources/xenocantoweb.png)


Las grabaciones están sujetas a distintas condiciones de uso. Para el objeto de nuestro estudio descargaremos las que no presenten limitaciones a su uso.

>Recording License
>Recordings on xeno-canto are licensed under a small number of different Creative Commons licenses. You can search for recordings that match specific license conditions using the lic tag. Possible license conditions include Attribution (BY), NonCommercial (NC), ShareAlike (SA), and NoDerivatives (ND). Conditions should be separated by a '-' character. For instance, to find recordings that are licensed under an Attribution-NonCommercial-ShareAlike license, use lic:BY-NC-SA. See the [Creative Commons website](https://creativecommons.org/licenses/) for more details about the individual licenses.


#### Criterios de búsqueda


No vamos a utilizar todas las grabaciones alojadas en la web, vamos a trabajar con una selección de esta. La web presenta la ventaja de realizar búsquedas avanzadas a través de [tags](https://www.xeno-canto.org/help/search)  incluidos en la url de petición, lo que permite realizar la selección de forma sencilla. Los criterios de búsqueda serán:

* **Calidad**. Las grabaciones están categorizadas en base a la calidad, desde la más alta A a la más baja E. Nosotros utilizaremos aquellas correspondientes a **A y B**.
* **Continente**. Grabaciones pertenecientes a **Europa**
* **Licencia**.- aquellas que no tienen limitaciones a su uso **BY-NC-SA**






## b) Webscrapping

Para obtener el dataset vamos a utilizar [**webscrapping**](https://es.wikipedia.org/wiki/Web_scraping), que es una técnica utilizada mediante programas de software para extraer información de sitios web. Usualmente, estos programas simulan la navegación de un humano en la World Wide Web ya sea utilizando el protocolo HTTP manualmente, o incrustando un navegador en una aplicación. 

Para ello, utilizaremos:


* [**Requests**](http://docs.python-requests.org/en/master/user/install/#install). Permite navegar y recuperar el contenido de páginas web.

>Requests is an elegant and simple HTTP library for Python, built for human beings. 

    conda install -c anaconda requests 

* [**Beautifulsoup4**](https://beautiful-soup-4.readthedocs.io/en/latest/). Permite extraer información del contenido HTML, XML de una página web recuperada.

>Beautiful Soup is a library for pulling data out of HTML and XML files. It provides ways of navigating, searching, and modifying parse trees.

    conda install -c anaconda beautifulsoup4 
    

## 1.- Librerías 

In [1]:
# librerías de uso común en el notebook
import requests
from bs4 import BeautifulSoup
import re
import datetime
import pandas as pd

## 2.- Funciones

Funciones que se encargan de recuperar la información del contenido de las páginas que vamos recuperando con las diferentes grabaciones. Con bs4 vamos tratando el HTML recuperado y obteniendo de él las distintas grabaciones que formarán parte del data set

### Parsea columnas

Recupera las columnas con los datos de la grabación a partir de una fila pasada por parámetro

In [21]:
#----------------------------------------------------------------------------
# webscraping_parse_columns(columns)
#  argumentos: 
#      column: columnas con los datos de la grabación
#  return: 
#      ls_columns: lista con la información de la grabación
#----------------------------------------------------------------------------
def webscraping_parse_columns(columns):
    # Lista temporal con la información de la grabación
    ls_columns = []
    
    # Validar previamente que la grabación se corresponde con un ave y tiene
    # un nombre común
    if columns[1].find("span", {"class": "common-name"}) == None:
        return ls_columns
    
    # Parsea columnas
    
    # Common-name
    common_name = columns[1].find("span", {"class": "common-name"})
    if common_name != None:
        ls_columns.append(common_name.text.strip())
        
    # Scientific-name
    scientific_name = columns[1].find("span", {"class": "scientific-name"})
    if scientific_name != None:
        ls_columns.append(scientific_name.text.strip())
        
    # Length
    ls_columns.append(columns[2].text.strip())
    
    # Recordist
    ls_columns.append(columns[3].text.strip())
    
    # Date
    ls_columns.append(columns[4].text.strip())
    
    # Country
    ls_columns.append(columns[6].text.strip())
    
    # Location
    ls_columns.append(columns[7].text.strip())
    
    # Type
    ls_columns.append(columns[9].text.strip())
    
    # retorna lista con la grabación parseada
    return ls_columns


### Parsea calidades de audio

Recupera los tipos de calidades de audio a partir del contenido HTML de una página web pasada por parámetro.

In [22]:
#----------------------------------------------------------------------------
# webscraping_parse_rows(bs4_content)
#  argumentos: 
#      bs4_content: contenido HTML de la página
#  return: 
#      classes: diccionario con los distintos tipos de calidades de audio
#----------------------------------------------------------------------------
def webscraping_parse_classes(bs4_content):
    # diccionario con las clases
    classes = {}

    # recuperar classes a partir del tag 'li'
    recordings = bs4_content.select('li[class="selected"]')
    
    # itera sobre las etiquetas recuperadas y va almacenando los distintos
    # tipos de classes en un diccionario
    for record in recordings:
        record_id = record.get('id')
        
        if record_id != None:
            ls_id = record_id.split("-")
            k_id = ls_id[1]
            v_classification = ls_id[2]
            classes[k_id] = v_classification
    
    return classes


### Parsea filas

Recupera cada una de las filas de la tabla HTML y va tratando cada una de ellas para recuperar las columnas correspondientes a las carácteristicas e identificación de la grabación.

In [23]:
#----------------------------------------------------------------------------
# webscraping_parse_rows(bs4_content)
#  argumentos: 
#      bs4_content: contenido HTML de la página
#  return: 
#      ls_rows: lista con las grabaciones
#----------------------------------------------------------------------------
def webscraping_parse_rows(bs4_content):
    # recuperar las calidades del audio en el HTML
    classes_selected =  webscraping_parse_classes(bs4_content)

    # las filas con información válida tienen 11 columnas
    COLUMNS_ITEMS = 11
    
    # recuperar las etiquetas de tipo fila "tr" en el documento HTML
    rows = bs4_content.select('tr')
    
    # lista temporal con las grabaciones
    ls_rows = []
    
    # recorrer la tabla HTML y recuperar los datos de la grabación por
    # cada una de las filas tratadas
    for row in rows:
        # lista temporal con las columnas de la grabación
        ls_columns = []
        
        # recuperar las etiquetas de columnas "td" 
        columns = row.select('td')
        
        # parsear datos si es una fila válida
        if len(columns) == COLUMNS_ITEMS:
            ls_columns = webscraping_parse_columns(columns)
        
        # añade grabación a lista
        if len(ls_columns) > 0:
            # recuperar ID de la grabación 
            id_= row.find("div", {"class": "jp-type-single"})
            
            if id_ != None:
                # añadir identificador de la grabación a la lista
                id_number = id_.attrs['data-xc-id'].strip()
                ls_columns.append('XC' + id_.attrs['data-xc-id'].strip())

                # añadir la clase seleccionada, y si no la localiza, el tipo es por defecto 0
                if id_number in classes_selected:
                    ls_columns.append(classes_selected[id_number])
                else:
                    ls_columns.append('0')
                
                # añade grabación a la lista
                ls_rows.append(ls_columns)

    # retorna lista con las grabaciones
    return ls_rows


### Parsea páginas

A partir del contenido HTML de una página web, recupera la tabla con la información de las grabaciones, y va recuperando cada una de ellas, hasta llegar al número de páginas pasado por parámetros.

In [24]:
#----------------------------------------------------------------------------
# webscraping_parse_pages(url_path, numpages)
#  argumentos: 
#      url_path: url de la web a scrapear
#      numpages: número de páginas a tratar 
#  return: 
#      ls_recordings: lista con las grabaciones
#----------------------------------------------------------------------------
def webscraping_parse_pages(url_path, numpages):
    print(">>>> scraping pages...")
    
    # status code ok (código de recuperación de página ok)
    PAGE_OK = 200
    
    # lista donde se almacena las grabaciones
    ls_recordings = []
    
    # recupera las páginas y extrae la información de ella
    for p in range(1, numpages):
        # mensaje por pantalla
        if p % 10 == 0:
            print(">>>> pages:", p, end='\r', flush=True)
        
        # lista temporal
        ls_temp = []
        
        # recuperar el contenido de la página web "p". Añadir
        # el número de la página en la URL
        page = requests.get(url_path.strip() + str(p))
        
        # parsear el contenido para recuperar las grabaciones
        if page.status_code == PAGE_OK:
            soup = BeautifulSoup(page.content, 'html.parser')
            ls_temp = webscraping_parse_rows(soup)
        else:
            print("Error recuperando página", str(p))
        
        # añadir grabaciones recuperadas
        if len(ls_temp) > 0: 
            ls_recordings = ls_recordings + ls_temp      

    #retorna lista con las grabaciones  
    return ls_recordings


### Normaliza nombre científico

El nombre cienfífico no viene normalizado, con el formato de "Genero especie"; en algunas ocasiones inclye la subespecie. Para poder utilizar luego el dataset y agrupar la información en base a la especie, la función devuelve sólo el Género más la especie.

In [25]:
#----------------------------------------------------------------------------
# webscraping_normalize_scientific_name(name)
#  argumento: 
#      name:nombre científico
#  return: 
#      name:nombre cienfífico normalizado (género + especie). 
#----------------------------------------------------------------------------
def webscraping_normalize_scientific_name(name):
    if len(name.split()) > 2:
        return name.split()[0] + ' ' + name.split()[1]
    else:
        return name
    

## 3.- Genera dataframe

Realiza el proceso de webscrapping de la web y crea un dataframe a partir de este. El dataframe consta de la siguiente información, por cada una de las grabaciones:

* **Common**: nombre común de la especie
* **Scientific**: nombre científico de la especie
* **Length**: duración de la grabación en formato HH:MM
* **Recordist**: persona que realiza la grabación
* **Country**: país donde se realiza la grabación
* **Location**: localidad donde se realiza la grabación
* **Type**: tipo de canto grabado (canto, llamada, vuelo, etc, etc)
* **ID**: identificador único de la grabación
* **Class**: calidad del audio
* **Seconds**: duración en número de segundos
* **Name**: nombre científico normalizado (Genero especie)



In [34]:
#----------------------------------------------------------------------------
# webscraping(url_path, max_numpages, csv_file):
# argumentos:
#     url_path.- url de la web a webscrapear
#     numpages.- número de páginas a tratar
#     csv_file .- fichero de salva del dataset
#----------------------------------------------------------------------------
def webscraping(url_path_search, numpages, csv_file):
    print(">>> Web scraping...")
    print(">>>", url_path_search)
    print(">>>> number of pages:", numpages)
    
    # Recupera páginas y genera una lista con las grabaciones
    ls_birdsongs = []
    ls_birdsongs = webscraping_parse_pages(url_path_search, numpages)
    
    # Crea un dataframe con las grabaciones
    if len(ls_birdsongs) > 0:
        # nombre de las columnas
        columns_names = ['Common','Scientific', 'Length', 'Recordist', 
                         'Date', 'Country', 'Location', 'Type','ID', 'Class']
        
        # dataframe
        print("\n>>> creating dataframe...")
        df_birdsongs = pd.DataFrame.from_records(ls_birdsongs, columns=columns_names)
        
        # crear columna con longitud en segundos
        df_birdsongs['Seconds'] = df_birdsongs['Length'].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]))
        
        # convertir la columna de fecha a formato fecha
        df_birdsongs['Date'] = df_birdsongs['Date'].apply(pd.to_datetime, format='%Y-%m-%d', errors='ignore')
        
        # crear columna con el nombre científico normalizado (genero + especie)
        df_birdsongs['Name'] =  df_birdsongs['Scientific'].map(webscraping_normalize_scientific_name)
        
        # salvar a fichero csv
        print(">>> saving dataframe...", csv_file)
        df_birdsongs.to_csv(csv_file, index= False)


## 4.- Scraping DataSet

Lanzamos el proceso de webscrapping de la página web y generamos un fichero csv con el resultado.

### Criterios de búqueda

#### Area

>The area tag allows you to search by world area. Valid values for this tag include africa, america, asia, australia, europe.

In [27]:
# Continente
area = 'europe'

#### Calidad

>Recording Quality
>Recordings are rated by quality. Quality ratings range from A (highest quality) to E (lowest quality). To search for recordings that match a certain quality rating, use the q, q<, and q> tags. For example:

>* q:A will return recordings with a quality rating of A.
>* q<:C will return recordings with a quality rating of D or E.
>* q>:C will return recordings with a quality rating of B or A.
>Note that not all recordings are rated. Unrated recordings will not be returned for a search on quality rating. To search explicitly for unrated recordings, use the special query q:0.


In [28]:
# Calidad de las grabaciones hasta "C" no incluida (q>:C)
quality = 'q>%3AC'

#### Licencia

>Recording License
>Recordings on xeno-canto are licensed under a small number of different Creative Commons licenses. You can search for recordings that match specific license conditions using the lic tag. Possible license conditions include Attribution (BY), NonCommercial (NC), ShareAlike (SA), and NoDerivatives (ND). Conditions should be separated by a '-' character. For instance, to find recordings that are licensed under an Attribution-NonCommercial-ShareAlike license, use lic:BY-NC-SA. See the Creative Commons website for more details about the individual licenses.

In [29]:
license = "BY-NC-SA"

### Scrapping!!

Lanza el proceso de scrapping...

In [30]:
%%time 

# Criterios de Búsqueda
# URL 
url_path = 'https://www.xeno-canto.org/'  

# URL con los criterios de búsqueda
url_path_search = url_path + \
                "explore?query=" + \
                "area%3A" + area + \
                "+" + quality + \
                "+lic%3A" + license + \
                "+&pg=" 

# Número de páginas a tratar (revisión manual de hasta donde llega el dataset, se podría automatizar)
numpages = 1527

# Nombre del fichero csv donde salvar el dataset
now = datetime.datetime.now()
csv_file = 'Birdsongs' + '_' + \
           area + '_' + \
           now.strftime("%Y%m%d%H%M%S") + '.csv'

# Creamos el dataset
webscraping(url_path_search, numpages, csv_file)


print(">>> Proceso completado!!!")
        

>>> web scraping...
>>> https://www.xeno-canto.org/explore?query=area%3Aeurope+q>%3AC+lic%3ABY-NC-SA+&pg=
>>>> number of pages: 1527
>>>> scraping pages...
>>>> pages: 1520
>>> creating dataframe...
>>> saving dataframe... Birdsongs_europe_q>%3AC_20190107181700.csv
>>> Proceso completado!!!
CPU times: user 3min 25s, sys: 1 s, total: 3min 26s
Wall time: 35min 14s
